In [21]:
import pymc3 as pm

def define_bart_model(x, y, num_trees=50, tree_depth=3, alpha=0.95):
    with pm.Model() as bart_model:
        num_trees = 50
        tree_depth = 3
        alpha = 0.95

    return bart_model
    


In [26]:
from ray import tune
config = {
    "learning_rate": tune.loguniform(0.001, 0.1),
    "batch_size": tune.choice([16, 32, 64, 128]),
}

In [27]:
def train_bart(config, x, y):
    # Obtener los valores de los hiperparámetros de la configuración de Ray Tune
    num_trees = config["num_trees"]
    tree_depth = config["tree_depth"]

    # Definir el modelo BART con los hiperparámetros actuales
    bart_model = define_bart_model(x, y, num_trees=num_trees, tree_depth=tree_depth)

    # Ajustar el modelo utilizando MCMC
    with bart_model:
        trace = pm.sample(2000, tune=1000)

    # Calcular el log-likelihood negativo como métrica para optimizar (menor es mejor)
    log_likelihood = -trace.log_likelihood.mean()

    return log_likelihood


In [29]:
import pandas as pd


if __name__ == "__main__":
    data = pd.read_csv('Datos_Bart_C.csv', delimiter=';')
    x = data[['total_credits_1', 'total_credits_2', 'total_courses_1', 'course_approved_1', 't_gpa_1']].values
    y = data['Permanencia_term_3'].values
    
    
    # Iniciar Ray 
ray.init()
    
    # Ejecutar Ray Tune
analysis = tune.run(
        tune.with_parameters(train_bart, x=x, y=y),
        config=config,
        metric="log_likelihood",
        mode="min",
        num_samples=10,
        resources_per_trial={"cpu": 1},
    )

    # Obtener los mejores hiperparámetros
best_config = analysis.get_best_config(metric="log_likelihood", mode="min")
print("Mejores hiperparámetros:", best_config)


RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.